In [1]:
import numpy as np
from astropy.io import fits
from marvin.tools import Maps
import marvin.utils.plot.colorbar as colorbar
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import scipy as scipy
from scipy.fft import fft
from scipy import ndimage

import multiprocessing as mp
from multiprocessing import Pool
import copy

import Simpler_Classifier

from importlib import reload 


[INFO]: No release version set. Setting default to DR17
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: Passing field metadata as keyword arguments is deprecated. Use the explicit `metadata=...` argument instead. Additional metadata: {'location': 'view_args'} (RemovedInMarshmallow4Warning)
[WARNING]: The 'missing' argument to fields is deprecated. Use 'load_default' instead. (RemovedInMarshmallow4Warning)


In [2]:
my_module = reload(Simpler_Classifier) # reload of Simpler_Classifier without restart

In [3]:
HEADER_PATH = '/Users/runquanguan/Documents/bicone_training_set.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']


In [ ]:
# detect from r = 0.3
# 

# measure the difference between before-gaussian and after-gaussian
# count number of "zero bin".
# measuring bin quality
# 